 Import Libraries

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pynance as pn
#import talib
import os

Load the Stock Data

In [5]:
stock_files = {
    "AAPL": "../data/stock_data/AAPL_historical_data.csv",
    "AMZN": "../data/stock_data/AMZN_historical_data.csv",
    "GOOG": "../data/stock_data/GOOG_historical_data.csv",
    "META": "../data/stock_data/META_historical_data.csv",
    "MSFT": "../data/stock_data/MSFT_historical_data.csv",
    "NVDA": "../data/stock_data/NVDA_historical_data.csv",
    "TSLA": "../data/stock_data/TSLA_historical_data.csv"
}

dfs = {}
for symbol, path in stock_files.items():
    df = pd.read_csv(path, parse_dates=["Date"])
    df.set_index("Date", inplace=True)
    dfs[symbol] = df